In [1]:
!pip install gradio langdetect googletrans==3.1.0a0 SpeechRecognition


Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install --upgrade pip
!pip install gradio langdetect googletrans==3.1.0a0 SpeechRecognition pydantic==1.10.0 opencv-python==4.6.0.66
!pip install pip uninstall ydata-profiling
!pip install gradio langdetect googletrans==3.1.0a0 SpeechRecognition pydantic==1.10.0
!pip install pyaudio
!pip install numpy sounddevice openai-whisper deepl python-dotenv


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement uninstall (from versions: none)
ERROR: No matching distribution found for uninstall


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/800.5 kB ? eta -:--:--
     ------------------------------------- 800.5/800.5 kB 11.5 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/884.2 kB ? eta -:--:--
   --------------------------------------- 884.2/884.2 kB 38.9 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803358 sha256=bebd3c508161ae1635010900fca

In [1]:
import sounddevice as sd
import numpy as np
import whisper
import deepl
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize Whisper model
model = whisper.load_model("base")

# Initialize DeepL translator
translator = deepl.Translator(os.getenv("DEEPL_API_KEY"))

def record_audio(duration=5, samplerate=16000):
    print("Recording... Speak now!")
    audio = sd.rec(int(samplerate * duration), samplerate=samplerate, channels=1, dtype='float32')
    sd.wait()
    return audio.flatten()

def transcribe_and_translate(audio):
    try:
        # Transcribe audio using Whisper
        result = model.transcribe(audio)
        original_text = result["text"]
        
        # Detect language
        detected_language = result["language"]
        
        print(f"Detected language: {detected_language}")
        print(f"Transcribed text: {original_text}")
        
        # Translate to English using DeepL
        if detected_language != "en":
            translation = translator.translate_text(original_text, target_lang="EN-US")
            translated_text = translation.text
        else:
            translated_text = original_text
        
        return f"Original ({detected_language}): {original_text}\nEnglish: {translated_text}"
    except Exception as e:
        return f"An error occurred: {str(e)}"

def main():
    print("Advanced Audio to English Text Translation Tool")
    print("Speak into your microphone when prompted.")
    print("Your speech will be recognized and translated to English.")
    print("Enter 'quit' to exit the program.")
    
    while True:
        user_input = input("\nPress Enter to start recording (5 seconds), or type 'quit' to exit: ")
        
        if user_input.lower() == 'quit':
            print("Thank you for using the translation tool. Goodbye!")
            break
        
        audio = record_audio()
        result = transcribe_and_translate(audio)
        print("\nResult:")
        print(result)

if __name__ == "__main__":
    main()

Advanced Audio to English Text Translation Tool
Speak into your microphone when prompted.
Your speech will be recognized and translated to English.
Enter 'quit' to exit the program.
Recording... Speak now!


C:\Users\austi\AppData\Roaming\Python\Python310\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Detected language: es
Transcribed text:  cómo está, señor?

Result:
Original (es):  cómo está, señor?
English:  How are you, sir?
Recording... Speak now!
Detected language: en
Transcribed text: 

Result:
Original (en): 
English: 


KeyboardInterrupt: Interrupted by user

In [3]:
!pip install tk sounddevice numpy whisper-openai deepl python-dotenv

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 1.2/1.2 MB 9.7 MB/s eta 0:00:00


In [2]:
import tkinter as tk
from tkinter import ttk, scrolledtext
import sounddevice as sd
import numpy as np
import whisper
import deepl
import os
from dotenv import load_dotenv
import threading
import time

# Load environment variables
load_dotenv()

# Initialize Whisper model
model = whisper.load_model("base")

# Initialize DeepL translator
translator = deepl.Translator(os.getenv("DEEPL_API_KEY"))

class TranslationApp:
    def __init__(self, master):
        self.master = master
        master.title("Language Translation Conversation App")
        master.geometry("800x600")

        self.create_widgets()
        self.is_recording = False
        self.current_language = "ja"  # Default to Japanese

    def create_widgets(self):
        # Language selection
        self.language_var = tk.StringVar(value="Japanese")
        language_label = ttk.Label(self.master, text="Select your language:")
        language_label.pack(pady=5)
        language_menu = ttk.Combobox(self.master, textvariable=self.language_var, values=["Japanese", "English"])
        language_menu.pack(pady=5)
        language_menu.bind("<<ComboboxSelected>>", self.change_language)

        # Text areas
        self.original_text = scrolledtext.ScrolledText(self.master, height=10)
        self.original_text.pack(pady=10, padx=10, fill=tk.BOTH, expand=True)

        self.translated_text = scrolledtext.ScrolledText(self.master, height=10)
        self.translated_text.pack(pady=10, padx=10, fill=tk.BOTH, expand=True)

        # Buttons
        self.start_button = ttk.Button(self.master, text="Start Recording", command=self.toggle_recording)
        self.start_button.pack(pady=5)

        self.translate_button = ttk.Button(self.master, text="Translate", command=self.translate_text)
        self.translate_button.pack(pady=5)

        self.clear_button = ttk.Button(self.master, text="Clear", command=self.clear_text)
        self.clear_button.pack(pady=5)

    def change_language(self, event):
        self.current_language = "ja" if self.language_var.get() == "Japanese" else "en"

    def toggle_recording(self):
        if not self.is_recording:
            self.is_recording = True
            self.start_button.config(text="Stop Recording")
            threading.Thread(target=self.record_audio, daemon=True).start()
        else:
            self.is_recording = False
            self.start_button.config(text="Start Recording")

    def record_audio(self):
        samplerate = 16000
        while self.is_recording:
            audio = sd.rec(int(samplerate * 5), samplerate=samplerate, channels=1, dtype='float32')
            sd.wait()
            audio = audio.flatten()
            self.transcribe_and_translate(audio)
            time.sleep(0.1)  # Small delay to prevent excessive CPU usage

    def transcribe_and_translate(self, audio):
        try:
            result = model.transcribe(audio)
            original_text = result["text"].strip()
            detected_language = result["language"]

            if original_text:
                self.original_text.insert(tk.END, f"{original_text}\n")
                self.original_text.see(tk.END)

                target_lang = "EN-US" if detected_language != "en" else "JA"
                translation = translator.translate_text(original_text, target_lang=target_lang)
                translated_text = translation.text

                self.translated_text.insert(tk.END, f"{translated_text}\n")
                self.translated_text.see(tk.END)

        except Exception as e:
            print(f"An error occurred: {str(e)}")

    def translate_text(self):
        original_text = self.original_text.get("1.0", tk.END).strip()
        if original_text:
            target_lang = "EN-US" if self.current_language == "ja" else "JA"
            translation = translator.translate_text(original_text, target_lang=target_lang)
            self.translated_text.delete("1.0", tk.END)
            self.translated_text.insert(tk.END, translation.text)

    def clear_text(self):
        self.original_text.delete("1.0", tk.END)
        self.translated_text.delete("1.0", tk.END)

if __name__ == "__main__":
    root = tk.Tk()
    app = TranslationApp(root)
    root.mainloop()

C:\Users\austi\AppData\Roaming\Python\Python310\site-packages\whisper\transcribe.py:78: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


KeyboardInterrupt: 

: 

In [3]:
import tkinter as tk
from tkinter import ttk, scrolledtext
import sounddevice as sd
import numpy as np
import whisper
import deepl
import os
from dotenv import load_dotenv
import threading
import queue

# Load environment variables
load_dotenv()

# Initialize Whisper model
model = whisper.load_model("base")

# Initialize DeepL translator
translator = deepl.Translator(os.getenv("DEEPL_API_KEY"))

class TranslationApp:
    def __init__(self, master):
        self.master = master
        master.title("Real-time Bilingual Conversation Translation")
        master.geometry("1000x800")

        self.create_widgets()
        self.is_listening = False
        self.audio_queue = queue.Queue()

    def create_widgets(self):
        # English section
        english_frame = ttk.LabelFrame(self.master, text="English")
        english_frame.pack(pady=10, padx=10, fill=tk.BOTH, expand=True)

        self.english_text = scrolledtext.ScrolledText(english_frame, height=10)
        self.english_text.pack(pady=5, padx=5, fill=tk.BOTH, expand=True)

        # Japanese section
        japanese_frame = ttk.LabelFrame(self.master, text="日本語")
        japanese_frame.pack(pady=10, padx=10, fill=tk.BOTH, expand=True)

        self.japanese_text = scrolledtext.ScrolledText(japanese_frame, height=10)
        self.japanese_text.pack(pady=5, padx=5, fill=tk.BOTH, expand=True)

        # Control buttons
        self.listen_button = ttk.Button(self.master, text="Start Listening", command=self.toggle_listening)
        self.listen_button.pack(pady=5)

        self.clear_button = ttk.Button(self.master, text="Clear All", command=self.clear_text)
        self.clear_button.pack(pady=5)

    def toggle_listening(self):
        if not self.is_listening:
            self.is_listening = True
            self.listen_button.config(text="Stop Listening")
            threading.Thread(target=self.audio_listener, daemon=True).start()
            threading.Thread(target=self.process_audio, daemon=True).start()
        else:
            self.is_listening = False
            self.listen_button.config(text="Start Listening")

    def audio_listener(self):
        def callback(indata, frames, time, status):
            if status:
                print(status, file=sys.stderr)
            self.audio_queue.put(bytes(indata))

        with sd.InputStream(samplerate=16000, channels=1, dtype='int16', callback=callback):
            while self.is_listening:
                sd.sleep(100)

    def process_audio(self):
        audio_data = b""
        silence_threshold = 500  # Adjust this value to set silence sensitivity
        silence_duration = 0
        max_silence = 1  # Maximum silence duration in seconds

        while self.is_listening:
            try:
                chunk = self.audio_queue.get(timeout=1)
                audio_data += chunk

                # Check for silence
                if np.abs(np.frombuffer(chunk, dtype=np.int16)).max() < silence_threshold:
                    silence_duration += len(chunk) / 32000  # 16000 sample rate * 2 bytes per sample
                else:
                    silence_duration = 0

                # Process audio if max silence is reached or buffer is large enough
                if silence_duration > max_silence or len(audio_data) > 32000 * 5:  # 5 seconds of audio
                    if len(audio_data) > 32000 * 0.5:  # Only process if there's at least 0.5 seconds of audio
                        audio_np = np.frombuffer(audio_data, dtype=np.int16).astype(np.float32) / 32768.0
                        self.transcribe_and_translate(audio_np)
                    audio_data = b""
                    silence_duration = 0

            except queue.Empty:
                pass

    def transcribe_and_translate(self, audio):
        try:
            result = model.transcribe(audio)
            original_text = result["text"].strip()
            detected_language = result["language"]

            if original_text:
                if detected_language == "en":
                    self.update_text(self.english_text, f"English: {original_text}\n")
                    translation = translator.translate_text(original_text, target_lang="JA")
                    self.update_text(self.japanese_text, f"翻訳: {translation.text}\n")
                elif detected_language == "ja":
                    self.update_text(self.japanese_text, f"日本語: {original_text}\n")
                    translation = translator.translate_text(original_text, target_lang="EN-US")
                    self.update_text(self.english_text, f"Translation: {translation.text}\n")
                else:
                    print(f"Detected unexpected language: {detected_language}")

        except Exception as e:
            print(f"An error occurred: {str(e)}")

    def update_text(self, text_widget, message):
        self.master.after(0, lambda: self.safe_update_text(text_widget, message))

    def safe_update_text(self, text_widget, message):
        text_widget.insert(tk.END, message)
        text_widget.see(tk.END)

    def clear_text(self):
        self.english_text.delete("1.0", tk.END)
        self.japanese_text.delete("1.0", tk.END)

if __name__ == "__main__":
    root = tk.Tk()
    app = TranslationApp(root)
    root.mainloop()

KeyboardInterrupt: 

In [4]:
!pip install wave

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wave: filename=Wave-0.0.2-py3-none-any.whl size=1229 sha256=2ed881fbbf1381e4838d5211a54c82a60a25eaf88594c2ce32d4761361ae0b2e
  Stored in directory: c:\users\austi\appdata\local\pip\cache\wheels\f8\24\4d\1b01c0e32da3eb3fd71bbbc6093fcc557ec3b2d9e532ecd65d
Successfully built wave


In [6]:
import tkinter as tk
from tkinter import ttk, scrolledtext
import sounddevice as sd
import numpy as np
import whisper
import deepl
import os
from dotenv import load_dotenv
import threading
import queue

# Load environment variables
load_dotenv()

# Initialize Whisper model
model = whisper.load_model("base")

# Initialize DeepL translator
translator = deepl.Translator(os.getenv("DEEPL_API_KEY"))

# DeepL supported languages
DEEPL_LANGUAGES = {
    "Bulgarian": "BG",
    "Czech": "CS",
    "Danish": "DA",
    "German": "DE",
    "Greek": "EL",
    "English": "EN",
    "Spanish": "ES",
    "Estonian": "ET",
    "Finnish": "FI",
    "French": "FR",
    "Hungarian": "HU",
    "Indonesian": "ID",
    "Italian": "IT",
    "Japanese": "JA",
    "Lithuanian": "LT",
    "Latvian": "LV",
    "Dutch": "NL",
    "Polish": "PL",
    "Portuguese": "PT",
    "Romanian": "RO",
    "Russian": "RU",
    "Slovak": "SK",
    "Slovenian": "SL",
    "Swedish": "SV",
    "Turkish": "TR",
    "Ukrainian": "UK",
    "Chinese": "ZH"
}

class TranslationApp:
    def __init__(self, master):
        self.master = master
        master.title("Multi-Language Real-time Conversation Translation")
        master.geometry("1200x800")

        self.create_widgets()
        self.is_listening = False
        self.audio_queue = queue.Queue()

    def create_widgets(self):
        # Language selection frame
        lang_frame = ttk.Frame(self.master)
        lang_frame.pack(pady=10, padx=10, fill=tk.X)

        # Source language selection
        ttk.Label(lang_frame, text="Source Language:").pack(side=tk.LEFT, padx=(0, 10))
        self.source_lang_var = tk.StringVar(value="English")
        self.source_lang_dropdown = ttk.Combobox(lang_frame, textvariable=self.source_lang_var, values=list(DEEPL_LANGUAGES.keys()), state="readonly")
        self.source_lang_dropdown.pack(side=tk.LEFT, padx=(0, 20))

        # Target language selection
        ttk.Label(lang_frame, text="Target Language:").pack(side=tk.LEFT, padx=(0, 10))
        self.target_lang_var = tk.StringVar(value="Japanese")
        self.target_lang_dropdown = ttk.Combobox(lang_frame, textvariable=self.target_lang_var, values=list(DEEPL_LANGUAGES.keys()), state="readonly")
        self.target_lang_dropdown.pack(side=tk.LEFT)

        # Source language text area
        source_frame = ttk.LabelFrame(self.master, text="Source")
        source_frame.pack(pady=10, padx=10, fill=tk.BOTH, expand=True)

        self.source_text = scrolledtext.ScrolledText(source_frame, height=10)
        self.source_text.pack(pady=5, padx=5, fill=tk.BOTH, expand=True)

        # Target language text area
        target_frame = ttk.LabelFrame(self.master, text="Target")
        target_frame.pack(pady=10, padx=10, fill=tk.BOTH, expand=True)

        self.target_text = scrolledtext.ScrolledText(target_frame, height=10)
        self.target_text.pack(pady=5, padx=5, fill=tk.BOTH, expand=True)

        # Control buttons
        self.listen_button = ttk.Button(self.master, text="Start Listening", command=self.toggle_listening)
        self.listen_button.pack(pady=5)

        self.clear_button = ttk.Button(self.master, text="Clear All", command=self.clear_text)
        self.clear_button.pack(pady=5)

    def toggle_listening(self):
        if not self.is_listening:
            self.is_listening = True
            self.listen_button.config(text="Stop Listening")
            threading.Thread(target=self.audio_listener, daemon=True).start()
            threading.Thread(target=self.process_audio, daemon=True).start()
        else:
            self.is_listening = False
            self.listen_button.config(text="Start Listening")

    def audio_listener(self):
        def callback(indata, frames, time, status):
            if status:
                print(status, file=sys.stderr)
            self.audio_queue.put(indata.copy())

        with sd.InputStream(samplerate=16000, channels=1, dtype='float32', callback=callback):
            while self.is_listening:
                sd.sleep(100)

    def process_audio(self):
        audio_buffer = np.array([], dtype=np.float32)
        silence_threshold = 0.01  # Adjust this value to set silence sensitivity
        silence_duration = 0
        max_silence = 1  # Maximum silence duration in seconds

        while self.is_listening:
            try:
                chunk = self.audio_queue.get(timeout=1)
                audio_buffer = np.concatenate((audio_buffer, chunk.flatten()))

                # Check for silence
                if np.abs(chunk).max() < silence_threshold:
                    silence_duration += len(chunk) / 16000
                else:
                    silence_duration = 0

                # Process audio if max silence is reached or buffer is large enough
                if silence_duration > max_silence or len(audio_buffer) > 16000 * 5:  # 5 seconds of audio
                    if len(audio_buffer) > 16000 * 0.5:  # Only process if there's at least 0.5 seconds of audio
                        self.transcribe_and_translate(audio_buffer)
                    audio_buffer = np.array([], dtype=np.float32)
                    silence_duration = 0

            except queue.Empty:
                pass

    def transcribe_and_translate(self, audio):
        try:
            # Transcribe using the audio numpy array
            result = model.transcribe(audio)
            original_text = result["text"].strip()
            detected_language = result["language"]

            if original_text:
                source_lang = DEEPL_LANGUAGES[self.source_lang_var.get()]
                target_lang = DEEPL_LANGUAGES[self.target_lang_var.get()]

                self.update_text(self.source_text, f"{self.source_lang_var.get()}: {original_text}\n")
                
                translation = translator.translate_text(original_text, target_lang=target_lang)
                self.update_text(self.target_text, f"{self.target_lang_var.get()}: {translation.text}\n")

        except Exception as e:
            print(f"An error occurred: {str(e)}")
            self.update_text(self.source_text, f"Error: {str(e)}\n")

    def update_text(self, text_widget, message):
        self.master.after(0, lambda: self.safe_update_text(text_widget, message))

    def safe_update_text(self, text_widget, message):
        text_widget.insert(tk.END, message)
        text_widget.see(tk.END)

    def clear_text(self):
        self.source_text.delete("1.0", tk.END)
        self.target_text.delete("1.0", tk.END)

if __name__ == "__main__":
    root = tk.Tk()
    app = TranslationApp(root)
    root.mainloop()

An error occurred: The size of tensor a (29) must match the size of tensor b (3) at non-singleton dimension 3


In [9]:
import tkinter as tk
from tkinter import ttk, scrolledtext
import sounddevice as sd
import numpy as np
import whisper
import deepl
import os
from dotenv import load_dotenv
import threading
import queue
import torch

# Load environment variables
load_dotenv()

# Initialize Whisper model (using a smaller model for faster inference)
model = whisper.load_model("tiny")

# Initialize DeepL translator
translator = deepl.Translator(os.getenv("DEEPL_API_KEY"))

# DeepL supported languages
DEEPL_LANGUAGES = {
    "Bulgarian": "BG", "Czech": "CS", "Danish": "DA", "German": "DE", "Greek": "EL",
    "English": "EN", "Spanish": "ES", "Estonian": "ET", "Finnish": "FI", "French": "FR",
    "Hungarian": "HU", "Indonesian": "ID", "Italian": "IT", "Japanese": "JA",
    "Lithuanian": "LT", "Latvian": "LV", "Dutch": "NL", "Polish": "PL", "Portuguese": "PT",
    "Romanian": "RO", "Russian": "RU", "Slovak": "SK", "Slovenian": "SL", "Swedish": "SV",
    "Turkish": "TR", "Ukrainian": "UK", "Chinese": "ZH"
}

class TranslationApp:
    def __init__(self, master):
        self.master = master
        master.title("Fast Multi-Language Real-time Conversation Translation")
        master.geometry("1200x800")

        self.create_widgets()
        self.is_listening = False
        self.audio_queue = queue.Queue()
        self.translation_queue = queue.Queue()

    def create_widgets(self):
        # Language selection frame
        lang_frame = ttk.Frame(self.master)
        lang_frame.pack(pady=10, padx=10, fill=tk.X)

        # Source language selection
        ttk.Label(lang_frame, text="Source Language:").pack(side=tk.LEFT, padx=(0, 10))
        self.source_lang_var = tk.StringVar(value="English")
        self.source_lang_dropdown = ttk.Combobox(lang_frame, textvariable=self.source_lang_var, values=list(DEEPL_LANGUAGES.keys()), state="readonly")
        self.source_lang_dropdown.pack(side=tk.LEFT, padx=(0, 20))

        # Target language selection
        ttk.Label(lang_frame, text="Target Language:").pack(side=tk.LEFT, padx=(0, 10))
        self.target_lang_var = tk.StringVar(value="Japanese")
        self.target_lang_dropdown = ttk.Combobox(lang_frame, textvariable=self.target_lang_var, values=list(DEEPL_LANGUAGES.keys()), state="readonly")
        self.target_lang_dropdown.pack(side=tk.LEFT)

        # Source language text area
        source_frame = ttk.LabelFrame(self.master, text="Source")
        source_frame.pack(pady=10, padx=10, fill=tk.BOTH, expand=True)

        self.source_text = scrolledtext.ScrolledText(source_frame, height=10)
        self.source_text.pack(pady=5, padx=5, fill=tk.BOTH, expand=True)

        # Target language text area
        target_frame = ttk.LabelFrame(self.master, text="Target")
        target_frame.pack(pady=10, padx=10, fill=tk.BOTH, expand=True)

        self.target_text = scrolledtext.ScrolledText(target_frame, height=10)
        self.target_text.pack(pady=5, padx=5, fill=tk.BOTH, expand=True)

        # Control buttons
        self.listen_button = ttk.Button(self.master, text="Start Listening", command=self.toggle_listening)
        self.listen_button.pack(pady=5)

        self.clear_button = ttk.Button(self.master, text="Clear All", command=self.clear_text)
        self.clear_button.pack(pady=5)

    def toggle_listening(self):
        if not self.is_listening:
            self.is_listening = True
            self.listen_button.config(text="Stop Listening")
            threading.Thread(target=self.audio_listener, daemon=True).start()
            threading.Thread(target=self.process_audio, daemon=True).start()
            threading.Thread(target=self.translate_text, daemon=True).start()
            threading.Thread(target=self.update_ui, daemon=True).start()
        else:
            self.is_listening = False
            self.listen_button.config(text="Start Listening")

    def audio_listener(self):
        def callback(indata, frames, time, status):
            if status:
                print(status, file=sys.stderr)
            self.audio_queue.put(indata.copy())

        with sd.InputStream(samplerate=16000, channels=1, dtype='float32', callback=callback, blocksize=1600):
            while self.is_listening:
                sd.sleep(10)

    def process_audio(self):
        audio_buffer = np.array([], dtype=np.float32)
        silence_threshold = 0.01
        silence_duration = 0
        max_silence = 0.5

        while self.is_listening:
            try:
                chunk = self.audio_queue.get(timeout=0.1)
                audio_buffer = np.concatenate((audio_buffer, chunk.flatten()))

                if np.abs(chunk).max() < silence_threshold:
                    silence_duration += len(chunk) / 16000
                else:
                    silence_duration = 0

                if silence_duration > max_silence or len(audio_buffer) > 16000 * 2:
                    if len(audio_buffer) > 16000 * 0.5:
                        self.transcribe_audio(audio_buffer)
                    audio_buffer = np.array([], dtype=np.float32)
                    silence_duration = 0

            except queue.Empty:
                pass

    def transcribe_audio(self, audio):
        with torch.no_grad():
            result = model.transcribe(audio, fp16=False)
        original_text = result["text"].strip()
        if original_text:
            self.translation_queue.put((original_text, self.source_lang_var.get(), self.target_lang_var.get()))

    def translate_text(self):
        while self.is_listening:
            try:
                original_text, source_lang, target_lang = self.translation_queue.get(timeout=0.1)
                source_lang_code = DEEPL_LANGUAGES[source_lang]
                target_lang_code = DEEPL_LANGUAGES[target_lang]
                
                translation = translator.translate_text(original_text, target_lang=target_lang_code)
                self.update_text(self.source_text, f"{source_lang}: {original_text}\n")
                self.update_text(self.target_text, f"{target_lang}: {translation.text}\n")
            except queue.Empty:
                pass

    def update_ui(self):
        while self.is_listening:
            self.master.update()
            self.master.after(10)

    def update_text(self, text_widget, message):
        self.master.after(0, lambda: self.safe_update_text(text_widget, message))

    def safe_update_text(self, text_widget, message):
        text_widget.insert(tk.END, message)
        text_widget.see(tk.END)

    def clear_text(self):
        self.source_text.delete("1.0", tk.END)
        self.target_text.delete("1.0", tk.END)

if __name__ == "__main__":
    root = tk.Tk()
    app = TranslationApp(root)
    root.mainloop()

100%|█████████████████████████████████████| 72.1M/72.1M [00:04<00:00, 18.6MiB/s]


In [12]:
###using Local Transformer for translation with the intention of increasing real time speed
!pip install tk sounddevice numpy torch openai-whisper transformers sentencepiece

Defaulting to user installation because normal site-packages is not writeable


In [13]:
import tkinter as tk
from tkinter import ttk, scrolledtext
import sounddevice as sd
import numpy as np
import whisper
import torch
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
import threading
import queue

# Initialize Whisper model (using a smaller model for faster inference)
whisper_model = whisper.load_model("tiny")

# Initialize M2M100 model and tokenizer
m2m_model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
m2m_tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

# Supported languages (M2M100 supports many more, but we'll use a subset for simplicity)
LANGUAGES = {
    "English": "en", "Spanish": "es", "French": "fr", "German": "de", "Italian": "it",
    "Portuguese": "pt", "Russian": "ru", "Chinese": "zh", "Japanese": "ja", "Korean": "ko",
    "Arabic": "ar", "Hindi": "hi", "Dutch": "nl", "Polish": "pl", "Turkish": "tr"
}

class TranslationApp:
    def __init__(self, master):
        self.master = master
        master.title("Fast Multi-Language Real-time Conversation Translation (Local)")
        master.geometry("1200x800")

        self.create_widgets()
        self.is_listening = False
        self.audio_queue = queue.Queue()
        self.translation_queue = queue.Queue()

    def create_widgets(self):
        # Language selection frame
        lang_frame = ttk.Frame(self.master)
        lang_frame.pack(pady=10, padx=10, fill=tk.X)

        # Source language selection
        ttk.Label(lang_frame, text="Source Language:").pack(side=tk.LEFT, padx=(0, 10))
        self.source_lang_var = tk.StringVar(value="English")
        self.source_lang_dropdown = ttk.Combobox(lang_frame, textvariable=self.source_lang_var, values=list(LANGUAGES.keys()), state="readonly")
        self.source_lang_dropdown.pack(side=tk.LEFT, padx=(0, 20))

        # Target language selection
        ttk.Label(lang_frame, text="Target Language:").pack(side=tk.LEFT, padx=(0, 10))
        self.target_lang_var = tk.StringVar(value="Japanese")
        self.target_lang_dropdown = ttk.Combobox(lang_frame, textvariable=self.target_lang_var, values=list(LANGUAGES.keys()), state="readonly")
        self.target_lang_dropdown.pack(side=tk.LEFT)

        # Source language text area
        source_frame = ttk.LabelFrame(self.master, text="Source")
        source_frame.pack(pady=10, padx=10, fill=tk.BOTH, expand=True)

        self.source_text = scrolledtext.ScrolledText(source_frame, height=10)
        self.source_text.pack(pady=5, padx=5, fill=tk.BOTH, expand=True)

        # Target language text area
        target_frame = ttk.LabelFrame(self.master, text="Target")
        target_frame.pack(pady=10, padx=10, fill=tk.BOTH, expand=True)

        self.target_text = scrolledtext.ScrolledText(target_frame, height=10)
        self.target_text.pack(pady=5, padx=5, fill=tk.BOTH, expand=True)

        # Control buttons
        self.listen_button = ttk.Button(self.master, text="Start Listening", command=self.toggle_listening)
        self.listen_button.pack(pady=5)

        self.clear_button = ttk.Button(self.master, text="Clear All", command=self.clear_text)
        self.clear_button.pack(pady=5)

    def toggle_listening(self):
        if not self.is_listening:
            self.is_listening = True
            self.listen_button.config(text="Stop Listening")
            threading.Thread(target=self.audio_listener, daemon=True).start()
            threading.Thread(target=self.process_audio, daemon=True).start()
            threading.Thread(target=self.translate_text, daemon=True).start()
            threading.Thread(target=self.update_ui, daemon=True).start()
        else:
            self.is_listening = False
            self.listen_button.config(text="Start Listening")

    def audio_listener(self):
        def callback(indata, frames, time, status):
            if status:
                print(status, file=sys.stderr)
            self.audio_queue.put(indata.copy())

        with sd.InputStream(samplerate=16000, channels=1, dtype='float32', callback=callback, blocksize=1600):
            while self.is_listening:
                sd.sleep(10)

    def process_audio(self):
        audio_buffer = np.array([], dtype=np.float32)
        silence_threshold = 0.01
        silence_duration = 0
        max_silence = 0.5

        while self.is_listening:
            try:
                chunk = self.audio_queue.get(timeout=0.1)
                audio_buffer = np.concatenate((audio_buffer, chunk.flatten()))

                if np.abs(chunk).max() < silence_threshold:
                    silence_duration += len(chunk) / 16000
                else:
                    silence_duration = 0

                if silence_duration > max_silence or len(audio_buffer) > 16000 * 2:
                    if len(audio_buffer) > 16000 * 0.5:
                        self.transcribe_audio(audio_buffer)
                    audio_buffer = np.array([], dtype=np.float32)
                    silence_duration = 0

            except queue.Empty:
                pass

    def transcribe_audio(self, audio):
        with torch.no_grad():
            result = whisper_model.transcribe(audio, fp16=False)
        original_text = result["text"].strip()
        if original_text:
            self.translation_queue.put((original_text, self.source_lang_var.get(), self.target_lang_var.get()))

    def translate_text(self):
        while self.is_listening:
            try:
                original_text, source_lang, target_lang = self.translation_queue.get(timeout=0.1)
                source_lang_code = LANGUAGES[source_lang]
                target_lang_code = LANGUAGES[target_lang]
                
                # Tokenize the input text
                m2m_tokenizer.src_lang = source_lang_code
                encoded_text = m2m_tokenizer(original_text, return_tensors="pt")

                # Generate translation
                generated_tokens = m2m_model.generate(**encoded_text, forced_bos_token_id=m2m_tokenizer.get_lang_id(target_lang_code))
                
                # Decode the generated tokens
                translation = m2m_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

                self.update_text(self.source_text, f"{source_lang}: {original_text}\n")
                self.update_text(self.target_text, f"{target_lang}: {translation}\n")
            except queue.Empty:
                pass
            except Exception as e:
                print(f"Translation error: {str(e)}")

    def update_ui(self):
        while self.is_listening:
            self.master.update()
            self.master.after(10)

    def update_text(self, text_widget, message):
        self.master.after(0, lambda: self.safe_update_text(text_widget, message))

    def safe_update_text(self, text_widget, message):
        text_widget.insert(tk.END, message)
        text_widget.see(tk.END)

    def clear_text(self):
        self.source_text.delete("1.0", tk.END)
        self.target_text.delete("1.0", tk.END)

if __name__ == "__main__":
    root = tk.Tk()
    app = TranslationApp(root)
    root.mainloop()

C:\Users\austi\AppData\Roaming\Python\Python310\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

C:\Users\austi\AppData\Roaming\Python\Python310\site-packages\transformers\generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
